# 1. Import Dependencies

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

## Dataset
This specific dataset seperates hand gestures into 4 different classes.

- stop
- onefingerup
- twofingerup
- thumbsup

The information about each hand gesture is contained into 42 features.

In [2]:
CSV_COLUMN_NAMES = np.arange(43)
CSV_COLUMN_NAMES = [str(item) for item in CSV_COLUMN_NAMES]
CSV_COLUMN_NAMES[42] = "Species"
#SPECIES = ['stop', 'onefingerup', 'twofingerup', 'thumbsup']
SPECIES = ['back_left', 'back_center', 'back_right', 'center_left', 'center_center', 'center_right', 'front_left', 'front_center', 'front_right']
CSV_PATH = os.path.join("src", "dataHandGesture", "file_0.csv")

In [3]:
df = pd.read_csv(CSV_PATH, names=CSV_COLUMN_NAMES, header=0)
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [4]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,Species
394,-28,76,14,75,43,59,54,34,61,16,...,-46,-50,19,-48,-2,-43,-17,-38,-27,1
401,-22,107,17,89,43,60,51,29,59,6,...,-52,-48,28,-52,2,-52,-13,-52,-27,1
1176,60,118,87,80,92,37,82,4,81,-23,...,-43,-18,56,-45,35,-62,21,-78,8,3
2119,-20,55,14,50,46,31,65,10,79,-6,...,-35,-47,27,-55,7,-61,-6,-64,-21,7
2576,-65,40,-69,23,-57,5,-46,-8,-39,-23,...,-19,-9,32,15,23,32,13,47,3,8


In [5]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
394,-28,76,14,75,43,59,54,34,61,16,...,-14,-46,-50,19,-48,-2,-43,-17,-38,-27
401,-22,107,17,89,43,60,51,29,59,6,...,-17,-52,-48,28,-52,2,-52,-13,-52,-27
1176,60,118,87,80,92,37,82,4,81,-23,...,-69,-43,-18,56,-45,35,-62,21,-78,8
2119,-20,55,14,50,46,31,65,10,79,-6,...,-31,-35,-47,27,-55,7,-61,-6,-64,-21
2576,-65,40,-69,23,-57,5,-46,-8,-39,-23,...,55,-19,-9,32,15,23,32,13,47,3


In [6]:
train.shape  # we have 120 entires with 4 features

(2159, 42)

## Input Function
Remember that nasty input function we created earlier. Well we need to make another one here! Fortunatly for us this one is a little easier to digest.

In [7]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [8]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='10', shape=(1,), default_value=None, dty

# 2. Building the Model
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from. Some options are listed below.

- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear coorespondence in our data.

In [9]:
MODEL_PATH = os.path.join("Tensorflow", "workspace", "models", "my_hand_gesture_model")

In [10]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between len(SPECIES) classes.
    n_classes=len(SPECIES),
    model_dir= MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Tensorflow\\workspace\\models\\my_hand_gesture_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [11]:
#handGesture = 5000
orientationSteps = 15000

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=orientationSteps)
# We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 33.105137, step = 0
INFO:tensorflow:global_step/sec: 108.336
INFO:tensorflow:loss = 8.565228, step = 100 (0.924 sec)
INFO:tensorflow:global_step/sec: 93.8913
INFO:tensorflow:loss = 5.2893457, step = 200 (1.066 sec)

## Export Model

In [12]:
# To export
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
export_path = classifier.export_saved_model(MODEL_PATH, export_input_fn, as_text=True);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: Tensorflow\workspace\models\my_hand_gesture_model\temp-1637194301\saved_model.pbtxt


## Load Model

In [13]:
# Loading the estimator
predict_fn = tf.saved_model.load(export_path).signatures['predict']
predict_fn

<ConcreteFunction pruned(examples) at 0x3B499D30>

In [14]:
# Convert input data into serialized Example strings.
examples = []
for index, row in test.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

# Convert from list to tensor
examples = tf.constant(examples)

print(test)
print(feature)

       0   1   2   3   4   5   6   7   8   9  ...  32  33  34  35  36  37  38  \
17    70  66  90  30  88  -3  78 -25  71 -44  ... -64  -5  -8  37 -36  31 -53   
18    69  66  90  30  88  -3  77 -25  71 -44  ... -63  -8  -7  37 -35  31 -52   
48    73  77  96  43  95   6  86 -19  82 -40  ... -68 -13  -6  39 -37  31 -57   
67    68  80  94  45  97   7  89 -18  85 -41  ... -67 -11 -11  39 -40  32 -59   
73    61  69  84  35  88   0  82 -24  77 -43  ... -61  -7 -13  35 -36  30 -53   
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..   
2666 -56  45 -21  43  10  29  36  16  59   8  ...   9 -35 -57  18 -55   3 -53   
2671 -46  46 -16  50  14  40  36  31  58  25  ...  12 -37 -49  12 -45  -5 -41   
2675 -46  44 -16  48  13  39  36  28  58  21  ...   4 -37 -51  13 -48  -5 -44   
2678 -40  39 -13  37  11  24  30  13  49   4  ...  10 -37 -46  18 -47   3 -46   
2679 -50  40 -24  40   3  30  23  19  40  10  ...  12 -41 -43   7 -35 -10 -29   

      39  40  41  
17    26

In [15]:
# make predictions of all testset
predictions = predict_fn(examples=examples)
print(predictions)

{'all_class_ids': <tf.Tensor: shape=(540, 9), dtype=int32, numpy=
array([[0, 1, 2, ..., 6, 7, 8],
       [0, 1, 2, ..., 6, 7, 8],
       [0, 1, 2, ..., 6, 7, 8],
       ...,
       [0, 1, 2, ..., 6, 7, 8],
       [0, 1, 2, ..., 6, 7, 8],
       [0, 1, 2, ..., 6, 7, 8]])>, 'logits': <tf.Tensor: shape=(540, 9), dtype=float32, numpy=
array([[20.16983   , 12.744728  ,  0.92184526, ..., 13.731694  ,
         2.8499362 ,  7.1916175 ],
       [19.839706  , 12.64948   ,  0.8199514 , ..., 13.389817  ,
         2.641858  ,  7.2383037 ],
       [19.946264  , 14.781376  ,  0.7403527 , ..., 15.996112  ,
         0.39413628,  6.8753643 ],
       ...,
       [ 1.407147  ,  3.1993446 , -2.412029  , ...,  1.1510116 ,
         3.2768528 ,  3.4076786 ],
       [ 2.0258775 ,  3.1212218 , -2.4171042 , ...,  2.4901698 ,
         4.9029927 ,  4.322276  ],
       [-0.15090987,  0.08673261,  0.18264063, ..., -0.03640832,
         0.13202493,  0.29542777]], dtype=float32)>, 'probabilities': <tf.Tensor: shape=(5

In [16]:
# print results
for idx, resultPred in enumerate(predictions["class_ids"]):
    class_id = resultPred[0]
    probability = predictions['probabilities'][idx][class_id]
    print(probability)
    print(f"\tPrediction is {SPECIES[class_id]} {100 * probability :.2f}%")
    print(f"\tExpected: {SPECIES[test_y.iloc[idx]]}")

tf.Tensor(0.960477, shape=(), dtype=float32)
	Prediction is back_left 96.05%
	Expected: back_left
tf.Tensor(0.9705987, shape=(), dtype=float32)
	Prediction is back_left 97.06%
	Expected: back_left
tf.Tensor(0.91976476, shape=(), dtype=float32)
	Prediction is back_left 91.98%
	Expected: back_left
tf.Tensor(0.9920622, shape=(), dtype=float32)
	Prediction is back_left 99.21%
	Expected: back_left
tf.Tensor(0.9839722, shape=(), dtype=float32)
	Prediction is back_left 98.40%
	Expected: back_left
tf.Tensor(0.99775285, shape=(), dtype=float32)
	Prediction is back_left 99.78%
	Expected: back_left
tf.Tensor(0.99809355, shape=(), dtype=float32)
	Prediction is back_left 99.81%
	Expected: back_left
tf.Tensor(0.996523, shape=(), dtype=float32)
	Prediction is back_left 99.65%
	Expected: back_left
tf.Tensor(0.99573106, shape=(), dtype=float32)
	Prediction is back_left 99.57%
	Expected: back_left
tf.Tensor(0.9950296, shape=(), dtype=float32)
	Prediction is back_left 99.50%
	Expected: back_left
tf.Tenso

## Evaluation

In [17]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-18T01:11:46
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.36502s
INFO:tensorflow:Finished evaluation at 2021-11-18-01:11:47
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.8537037, average_loss = 0.551091, global_step = 15000, loss = 0.83309764
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000

Test set accuracy: 0.854



## Predictions
Now that we have a trained model it's time to use it to make predictions. I've written a little script below that allows you to type the features of a flower and see a prediction for its class.

In [18]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = CSV_COLUMN_NAMES[:-1]
predict = {}

for rowNum in range(5):
    expected = SPECIES[test_y.iloc[rowNum]]
    for idx, feature in enumerate(features):
      predict[feature] = [test.iloc[rowNum][idx]] #to predict the first row of test

    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print("\n")
        print('Prediction is "{}" ({:.1f}%)'.format(
            SPECIES[class_id], 100 * probability))
        print(f"Expected is {expected}")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "back_left" (96.0%)
Expected is back_left
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "back_left" (97.1%)
Expected is back_left
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done r